In [127]:
# Install geocoder
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge geocoder --yes

Solving environment: | 
  - anaconda::ca-certificates-2019.8.28-0, anaconda::certifi-2019.9.11-py37_0, anaconda::openssl-1.1.1d-h1de35cc_2
  - anaconda::certifi-2019.9.11-py37_0, anaconda::openssl-1.1.1d-h1de35cc_2, defaults::ca-certificates-2019.8.28-0
  - anaconda::ca-certificates-2019.8.28-0, anaconda::openssl-1.1.1d-h1de35cc_2, defaults::certifi-2019.9.11-py37_0
  - anaconda::openssl-1.1.1d-h1de35cc_2, defaults::ca-certificates-2019.8.28-0, defaults::certifi-2019.9.11-py37_0
  - anaconda::ca-certificates-2019.8.28-0, defaults::certifi-2019.9.11-py37_0, defaults::openssl-1.1.1d-h1de35cc_2
  - defaults::ca-certificates-2019.8.28-0, defaults::certifi-2019.9.11-py37_0, defaults::openssl-1.1.1d-h1de35cc_2
  - anaconda::ca-certificates-2019.8.28-0, anaconda::certifi-2019.9.11-py37_0, defaults::openssl-1.1.1d-h1de35cc_2
  - anaconda::certifi-2019.9.11-py37_0, defaults::ca-certificates-2019.8.28-0, defaults::openssl-1.1.1d-h1de35ccdone

# All requested packages already installed.

Solving 

In [128]:
import pandas as pd
import numpy as np
import geocoder

# Import geocoder
from geopy.geocoders import Nominatim

# Inport JSON to Data Frame Convertor
from pandas.io.json import json_normalize

In [129]:
# Load the dataframe file produced in capstone wk3 part 1 notebook
df = pd.read_csv("part1.csv")
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [130]:
# Function to geocode each row and return a tuple of the lat lng coordinates
# Note function consistently times out even with the while loop retry
def geocode_postcode(postcode):
    
    latlng = None
    max_retries = 3
    iteration = 0
    print("Trying Postcode {}".format(postcode))
    
    while(latlng is None and iteration < max_retries):
        g = geocoder.google('{}, Toronto, Ontario'.format(postcode))
        latlng = g.latlng
        iteration = iteration + 1
        
    return (latlng) 

In [131]:
# Apply the function to each row and add the returned tuple as a new column
#df["LatLng"] = df["Postalcode"].apply(geocode_postcode)

# Split the Tuple Column into separate latitude and longitude coordinates
#df[["Lat", "Lng"]] = pd.DataFrame(df["LatLng"].to_list(), index=df.index)

# Drop the latlng column to create the required dataframne
#df.drop(["LatLng"], axis=1)


Status code Unknown from https://maps.googleapis.com/maps/api/geocode/json: ERROR - HTTPSConnectionPool(host='maps.googleapis.com', port=443): Max retries exceeded with url: /maps/api/geocode/json?address=M1B%2C+Toronto%2C+Ontario&bounds=&components=&region=&language= (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x11c3b8350>, 'Connection to maps.googleapis.com timed out. (connect timeout=5.0)'))

In [132]:
# Try loading from file instead as life is too short
df_geo = pd.read_csv("geospatial_coordinates.csv")

print(df.shape)
print(df_geo.shape)
print(df.columns)
print(df_geo.columns)

# Join the data frames together by appending columns using a common key
df_joined = pd.merge(df, df_geo, how="left", left_on="PostalCode", right_on="Postal Code")


(103, 3)
(103, 3)
Index(['PostalCode', 'Borough', 'Neighborhood'], dtype='object')
Index(['Postal Code', 'Latitude', 'Longitude'], dtype='object')


In [133]:
df_joined = df_joined.drop(["Postal Code"], axis=1)
df_joined.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [135]:
# Show the row and column counts oif the data frame
df_joined.shape

(103, 5)

In [136]:
# Save the data frame so we can easily load it in future exercises
# Save into the current working directory
export = df_joined.to_csv("part2.csv", header=True, index=False)